In [12]:
import pandas as pd


data= pd.read_csv('diabetes.csv')
df = data.copy(deep = True)
df
df1 = df.select_dtypes(exclude=[object])
correlation = df1.corr()
round(correlation,3)
cor_target = abs(correlation["Outcome"])
relevent_features = cor_target[cor_target >= 0.20 ]
df = df[relevent_features.index]
print(df.shape)
df.columns

(768, 5)


Index(['Pregnancies', 'Glucose', 'BMI', 'Age', 'Outcome'], dtype='object')

In [13]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 1 )

In [14]:
import keras as kr
model = kr.models.Sequential()
model.add(kr.layers.Dense(128,input_dim=4,activation="relu"))
model.add(kr.layers.Dense(64,activation="relu"))
model.add(kr.layers.Dense(32,activation="relu"))
model.add(kr.layers.Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=100)

Epoch 1/100
39/39 [==============================] - 0s 8ms/step - loss: 0.9795 - accuracy: 0.5798
Epoch 2/100
39/39 [==============================] - 0s 5ms/step - loss: 0.6990 - accuracy: 0.6336
Epoch 3/100
39/39 [==============================] - 0s 5ms/step - loss: 0.8106 - accuracy: 0.5749
Epoch 4/100
39/39 [==============================] - 0s 5ms/step - loss: 0.8953 - accuracy: 0.5391
Epoch 5/100
39/39 [==============================] - 0s 5ms/step - loss: 0.7053 - accuracy: 0.5961
Epoch 6/100
39/39 [==============================] - 0s 5ms/step - loss: 0.6636 - accuracy: 0.6270
Epoch 7/100
39/39 [==============================] - 0s 5ms/step - loss: 0.6622 - accuracy: 0.6368
Epoch 8/100
39/39 [==============================] - 0s 5ms/step - loss: 0.6359 - accuracy: 0.6629
Epoch 9/100
39/39 [==============================] - 0s 5ms/step - loss: 0.7018 - accuracy: 0.6173
Epoch 10/100
39/39 [==============================] - 0s 5ms/step - loss: 0.7619 - accuracy: 0.6303
Epoch 11/

39/39 [==============================] - 0s 5ms/step - loss: 0.5161 - accuracy: 0.7557
Epoch 84/100
39/39 [==============================] - 0s 4ms/step - loss: 0.5460 - accuracy: 0.7296
Epoch 85/100
39/39 [==============================] - 0s 5ms/step - loss: 0.5251 - accuracy: 0.7573
Epoch 86/100
39/39 [==============================] - 0s 4ms/step - loss: 0.5121 - accuracy: 0.7508
Epoch 87/100
39/39 [==============================] - 0s 5ms/step - loss: 0.5306 - accuracy: 0.7524
Epoch 88/100
39/39 [==============================] - 0s 4ms/step - loss: 0.5274 - accuracy: 0.7590
Epoch 89/100
39/39 [==============================] - 0s 4ms/step - loss: 0.5134 - accuracy: 0.7704
Epoch 90/100
39/39 [==============================] - 0s 5ms/step - loss: 0.5082 - accuracy: 0.7459
Epoch 91/100
39/39 [==============================] - 0s 5ms/step - loss: 0.5049 - accuracy: 0.7459
Epoch 92/100
39/39 [==============================] - 0s 5ms/step - loss: 0.5113 - accuracy: 0.7557
Epoch 93/100


In [15]:
y_pred = model.predict(x_test)
y_pred = y_pred.round()


In [16]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 2ms/step - loss: 1.1510 - accuracy: 0.7403


[1.1510037183761597, 0.7402597665786743]

In [87]:
model.save('annmodel')

INFO:tensorflow:Assets written to: annmodel/assets


In [ ]:
model1 = kr.models.load_model('/content/annmodel.pkl')

In [ ]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.8667 - accuracy: 0.7922


[0.8667243719100952, 0.7922077775001526]

In [ ]:
import keras

import numpy as np
from flask import Flask,request,jsonify,render_template
app = Flask(__name__)
model = keras.models.load_model('annmodel', compile = False)
@app.route('/')
def home():
  return render_template('api.html')
@app.route('/predict',methods=['POST'])
def predict():
    int_features = [[float(x) for x in request.form.values()]]
    final_features = [[np.array(int_features)]]
    prediction = model.predict(int_features)

    return render_template('api.html',prediction_text='chance for diabetes : {} '.format(prediction))

if __name__ == '__main__':
    # Threaded option to enable multiple instances for multiple user access support
    app.run(threaded=True, port=5000)

In [ ]:
!python app.py